In [3]:
from wordcloud import WordCloud
from PIL import Image, ImageDraw
import numpy as np

In [5]:
# Step 1: Generate WordCloud on the mask
def generate_wordcloud_on_mask(word_list, image_path, font_path="Arial.ttf", image_size = (800, 600)):
    """
    Generate a WordCloud image on the mask created from the image without a background.

    Args:
    word_list (list): List of words for the WordCloud.
    image_path (str): Path to the image without a background.
    font_path (str, optional): Path to the font file. Defaults to "Arial.ttf".

    Returns:
    PIL.Image.Image: The WordCloud image.
    """
    # Load the image without a background
    image = Image.open(image_path).convert("RGBA")#.resize((image_size[0], image_size[1]))
    
    # Get the size of the primary image
    width, height = image.size

    # Convert image to numpy array
    np_image = np.array(image)

    # Create a mask from the alpha channel of the image
    mask = np_image[:, :, 3]

    # Generate WordCloud on the mask
    text = ' '.join(word_list)
    wordcloud = WordCloud(mask=mask, font_path=font_path, background_color=None, mode="RGBA", color_func=lambda *args, **kwargs: (255, 255, 255)).generate(text)
    
    # Convert WordCloud object to image
    wordcloud_image = wordcloud.to_image()
    
    # Resize WordCloud image to match the size of the primary image
    wordcloud_resized = wordcloud_image.resize((width, height))
    
    return wordcloud_resized

# Step 2: Load the primary image
def load_image(image_path, image_size=(800, 600)):
    """
    Load the image and resize it to the specified dimensions.

    Args:
    image_path (str): Path to the image.
    width (int): Width of the image.
    height (int): Height of the image.

    Returns:
    PIL.Image.Image: The loaded and resized image.
    """
    return Image.open(image_path)#.resize((image_size[0], image_size[1]))

# Step 3: Blend two images together
def blend_images(primary_image, without_bg_image, alpha=0.2):
    """
    Blend two images together.

    Args:
    primary_image (PIL.Image.Image): The primary image with background.
    without_bg_image (PIL.Image.Image): The image with background removed.

    Returns:
    PIL.Image.Image: The blended image.
    """
    # Ensure both images have the same mode
    primary_image = primary_image.convert('RGBA')
    without_bg_image = without_bg_image.convert('RGBA')
    
    # Ensure both images have the same size
    without_bg_image = without_bg_image.resize(primary_image.size)
    
    # Blend images
    return Image.blend(primary_image, without_bg_image, alpha=alpha)

# Example usage:
word_list = ["Example", "text", "for", "WordCloud", "overlay"]
font_path = "Arial.ttf"
primary_path = "./reena_1.jpg"
without_bg_path = "./reena_1-removebg.png"

# Load primary image
primary_image = load_image(primary_path)
without_bg_image = load_image(without_bg_path)

# Generate WordCloud on the mask of the image without background
wordcloud_image = generate_wordcloud_on_mask(word_list, without_bg_path)

# Blend primary image and WordCloud image
blended_image = blend_images(primary_image, wordcloud_image)
blended_image = blend_images(blended_image, without_bg_image, 0.1)

blended_image.convert("RGB").save('image_blended.jpg')
# Save or display the blended image
blended_image.show()
# blended_image.save("blended_image.png")
